In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from rlhfutils.modeling_override import set_forward
from torch.nn import CrossEntropyLoss
from rlhfutils.rewards import computelike
from datasets import Dataset
import pickle
from nltk import word_tokenize
import matplotlib.pyplot as plt
import random

In [31]:
l = CrossEntropyLoss(reduction='none')

In [52]:
a = torch.tensor([[[1, .9, 0],[1, 0, 1]], [[0, 1, 0],[0, 0, 1]]])
labs = torch.tensor([[1,2], [0,1]], dtype=torch.long)

In [53]:
a

tensor([[[1.0000, 0.9000, 0.0000],
         [1.0000, 0.0000, 1.0000]],

        [[0.0000, 1.0000, 0.0000],
         [0.0000, 0.0000, 1.0000]]])

In [57]:
l(a.view(-1, 3), labs.view(-1)).reshape(2,2).mean(dim=-1)

tensor([0.8915, 1.5514])

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
mod = AutoModelForCausalLM.from_pretrained("facebook/opt-125m", device_map=0).eval()
tok = AutoTokenizer.from_pretrained("facebook/opt-125m")


/u/prasanns/miniconda3/envs/rlhfenv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
bmod = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b", device_map=0).eval()
btok = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [6]:
distprefs = Dataset.load_from_disk("../../data/contrastivedistill/wikidatasft/")

In [6]:
def getqs(ex):
    ex['question'] = tok.decode(tok(ex['outputs']).input_ids[:12], skip_special_tokens=True).strip()
    ex['question'] = ex['question'].rsplit(" ", 1)[0]
    return ex

In [17]:
distprefs =distprefs.map(getqs, num_proc=20)

Loading cached processed dataset at /scratch/cluster/prasanns/research/rlhf-length-biases/data/contrastivedistill/wikidatasft/cache-e036645cced4c69e_*_of_00020.arrow


In [26]:
# distprefs.save_to_disk("../../data/contrastivedistill//")

In [43]:
# tmp = Dataset.from_file("../../data/contrastivedistill/contrastdistillprefs/cache-c467cac8d01228b8.arrow")

In [20]:
tmpdata['outputs'][6]

'Autism is a disease.\nI think you mean autism.\nI think you mean autism'

In [7]:
tmpdata = Dataset.load_from_disk("../../data/contrastivedistill/optsmallwiki/")

In [69]:
teststrs = ["hi there", "hi there"]
inps = tok(teststrs, return_tensors="pt").to(mod.device)
out = mod(**inps, labels=inps.input_ids, )

modified forward !
torch.Size([2, 2, 50272])


In [21]:
scos = computelike(tmpdata['outputs'], btok, bmod, tok, mod, pbar=True, process=False, bsize=32)

  0%|                                                                                                                                                                                        | 0/5295 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5295/5295 [10:37<00:00,  8.30it/s]


In [26]:
from tqdm import tqdm

In [27]:
def outstoprefs(outs, scos): 
    pdata = []
    for i in tqdm(range(0, len(outs), 2)):
        tmp = {
            'question':"",
        }
        stmp = scos[i:i+2]
        rtmp = outs[i:i+2]
        jind = 0 if stmp[0]>stmp[1] else 1
        tmp['score_j'] = stmp[jind]
        tmp['score_k'] = stmp[1-jind]
        tmp['response_j'] = rtmp[jind]
        tmp['response_k'] = rtmp[1-jind]
        tmp['magnitude'] = tmp['score_j'] - tmp['score_k']
        pdata.append(tmp)
    return Dataset.from_list(pdata)

In [28]:
optprefs = outstoprefs(tmpdata['outputs'], scos)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84720/84720 [00:00<00:00, 1164985.13it/s]


In [34]:
# optprefs.save_to_disk("../../data/contrastivedistill/contoptprefs")

In [35]:
tmp = Dataset.load_from_disk("../../data/contrastivedistill/contdfixed/")

In [93]:
with open('data/kscos.pkl', 'wb') as f:
    pickle.dump(scos, f)

In [49]:
with open('data/kscos.pkl', 'rb') as f:
    scosk = pickle.load(f)
with open('data/jscos.pkl', 'rb') as f:
    scosj = pickle.load(f)
    

In [55]:
base = 10000

In [56]:
scos = computelike(tmp['chosen'][base:base+100], btok, bmod, tok, mod, pbar=True, process=False, bsize=32)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  9.58it/s]


In [57]:
scosj[base:base+10]

[1.7853946685791016,
 1.675436019897461,
 1.3695907592773438,
 1.775179386138916,
 0.9523935317993164,
 1.6250791549682617,
 1.4206819534301758,
 1.7454352378845215,
 1.38377046585083,
 2.193281650543213]

In [60]:
len(tmp)

158400

In [66]:
tmp = tmp.add_column("scos_j", scosj[:len(tmp)])
tmp = tmp.add_column("scos_k", scosk[:len(tmp)])

In [63]:
tmp = tmp.rename_columns({"prompt":"question", 'chosen':'response_j', 'rejected':'response_k'})

In [67]:
tmp = tmp.select_columns(['question', 'response_j', 'response_k', 'scos_j', 'scos_k'])

In [68]:
tmp

Dataset({
    features: ['question', 'response_j', 'response_k', 'scos_j', 'scos_k'],
    num_rows: 158400
})

In [72]:
tmp = tmp.filter(lambda ex: ex['scos_j']>ex['scos_k'], num_proc=20)

In [81]:
tmp.save_to_disk("../../data/contrastivedistill/contdfixed")

In [78]:
def remvals(ex):
    ex['question']=""
    return ex

In [79]:
tmp = tmp.map(remvals, num_proc=10)

In [80]:
tmp[0]

{'question': '',
 'response_j': 'Anarchism is where libertarian communist meet anarchism ;) this could totally happen but first anarchacap',
 'response_k': "Anarchism is a form of radicalism.\nI'm not sure what you mean.",
 'scos_j': 1.5447883605957031,
 'scos_k': -0.09044671058654785}

In [84]:
tmp.filter(lambda ex: ex['scos_j']<1.5)

Dataset({
    features: ['question', 'response_j', 'response_k', 'scos_j', 'scos_k'],
    num_rows: 60180
})

In [86]:
tmp.save_to_disk("../../data/contrastivedistill/truncdata60k")

In [87]:
def getfrac(dset, exs=1000, log=True): 
    inds = list(range(len(dset)))
    sampinds = random.sample(inds, exs)
    if log:
        print(sampinds[:20])
    return dset.select(sampinds)

In [90]:
getfrac(tmp, 60000).save_to_disk("../../data/contrastivedistill/samp60k")

[27028, 8895, 23800, 24137, 80423, 92752, 96701, 81698, 77319, 117925, 33492, 34631, 156577, 131038, 42988, 21776, 116962, 78083, 76070, 62070]


In [91]:
heldnew = Dataset.load_from_disk("../../data/contrastivedistill/contrastdistillheldoutprefs/")

In [94]:
heldnew = heldnew.map(remvals, num_proc=5)

In [97]:
heldnew.save_to_disk("../../data/contrastivedistill/heldoutprefs/")